<a href="https://colab.research.google.com/github/Neo-glitch/t.f-2.0-practice/blob/master/Tf_Data_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-data-validation

     |████████████████████████████████| 1.4 MB 15.0 MB/s 
     |████████████████████████████████| 99 kB 8.6 MB/s 
     |████████████████████████████████| 19.2 MB 159 kB/s 
     |████████████████████████████████| 10.2 MB 43.5 MB/s 
     |████████████████████████████████| 23.6 MB 1.2 MB/s 
     |████████████████████████████████| 294 kB 47.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 253 kB 73.4 MB/s 
     |████████████████████████████████| 2.3 MB 59.2 MB/s 
     |████████████████████████████████| 508 kB 83.2 MB/s 
     |████████████████████████████████| 151 kB 68.5 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 267 kB 75.3 MB/s 
     |████████████████████████████████| 255 kB 66.1 MB/s 
     |████████████████████████████████| 435 kB 69.9 MB/s 
     |████████████████████████████████| 183 kB 58.7 MB/s 
     |████████████████████████████████| 180 kB 68.6 MB/s 
     |█████████████

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_data_validation as tfdv


# for printing in python 2
# from __future__ import print_function

#### Dataset analysis

In [ ]:
dataset = pd.read_csv("pollution_small.csv")

dataset.shape

(2188, 5)

In [ ]:
training_data = dataset[:1600]

training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [ ]:
test_set = dataset[1600:]

test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


### Data Analysis and Validation with TFDV

In [ ]:
# gen training data stats
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

In [ ]:
# infering the schema
schema = tfdv.infer_schema(train_stats)

In [ ]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


In [ ]:
test_stats = tfdv.generate_statistics_from_dataframe(test_set)

### Compare Test Stats and Schema

In [ ]:
anomalies = tfdv.validate_statistics(statistics = test_stats, schema = schema )

# display anomalies if any
tfdv.display_anomalies(anomalies)

In [ ]:
"""
anomalies e.g
- having a value lower than 10 and having a value higher than 10 as input
- Cast error when what is expected and what is passed
- Integer value smaller than 0
"""

'\nanomalies e.g\n- having a value lower than 10 and having a value higher than 10 as input\n- Cast error when what is expected and what is passed\n- Integer value smaller than 0\n'

In [ ]:
# new data with anomalies, i.e without target col
test_set_copy = test_set.copy()
test_set_copy.drop("soot", axis = 1, inplace =True)

In [ ]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(test_set_copy)

anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema = schema)

tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


In [ ]:
schema.default_environment.append("TRAINING")  # training env

# serving time env, since training col won't be there in serving since we are predicting here
schema.default_environment.append("SERVING")   

In [ ]:
# Removing target col from serving env schema but not in training
tfdv.get_feature(schema = schema, feature_path="soot").not_in_environment.append("SERVING")

In [ ]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, "SERVING")

tfdv.display_anomalies(serving_env_anomalies)

In [ ]:
# Freezing Schema for later use in end to end pipeline of more data val
tfdv.write_schema_text(schema = schema, output_path="pollution_schema.pbtxt")